In [1]:
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random
import numpy as np
import pandas as pd
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import glob
from gensim.models.phrases import Phrases, Phraser

In [2]:
path_train_unsup=r'C:\Users\raja4\Downloads\aclImdb_v1.tar\aclImdb\train\unsup'
file_list_unsup = glob.glob(path_train_unsup + "/*.txt")
unsup_reviews=[]
for i in range(0,len(file_list_unsup)):
    data=pd.read_table(file_list_unsup[i])
    unsup_reviews.append(data.columns.tolist()[0])

In [3]:
df=pd.DataFrame(unsup_reviews)

In [4]:
df=df.rename(columns={'0':'reviews'})

In [42]:
df

,0
0,"[i, admit, the, great, majority, of, films, re..."
1,"[take, low, budget, inexperienced, actors, dou..."
2,"[everybody, has, seen, back, to, the, future, ..."
3,"[doris, day, was, an, icon, of, beauty, in, si..."
4,"[after, series, of, silly, fun, loving, movies..."
...,...
49995,"[delightfully, awful, made, by, david, giancol..."
49996,"[watching, time, chasers, it, obvious, that, i..."
49997,"[at, the, beginning, we, can, see, members, of..."
49998,"[the, movie, was, incredible, ever, since, saw..."


In [5]:
def text_to_word_list(text):
    """
    Preprocess and convert texts to a list of words 
    """
    text = str(text)
    text = text.lower()
    
    # remove html tags
    text = re.sub(r'<[^>]+>', '', text)
    # remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Single character removal
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    text = text.split()
    
    return text

In [6]:
df[0] = df[0].apply(lambda x: text_to_word_list(x))

In [7]:
#preparation pour Word2vec
text = [row for row in df[0]]
phrases = Phrases(text, min_count=1)
bigram = Phraser(phrases)
sentences = bigram[text]
sentences[0]

['i',
 'admit',
 'the',
 'great',
 'majority',
 'of',
 'films',
 'released',
 'before',
 'say',
 'are',
 'just',
 'not',
 'for',
 'me',
 'of',
 'the',
 'dozen',
 'or',
 'so',
 'major',
 'silents',
 'have',
 'viewed',
 'one',
 'loved',
 'the',
 'crowd',
 'and',
 'two',
 'were',
 'very',
 'good',
 'the',
 'last',
 'command',
 'and',
 'city_lights',
 'that',
 'latter',
 'chaplin',
 'circa',
 'so',
 'was',
 'apprehensive_about',
 'this',
 'one',
 'and',
 'humor',
 'is',
 'often',
 'difficult',
 'to',
 'appreciate',
 'uh',
 'enjoy',
 'decades_later',
 'did',
 'like',
 'the',
 'lead',
 'actors',
 'but',
 'thought',
 'little',
 'of',
 'the',
 'film',
 'one',
 'intriguing',
 'sequence',
 'early',
 'on',
 'the',
 'guys',
 'are',
 'supposed',
 'to',
 'get',
 'de',
 'loused',
 'and',
 'for',
 'about',
 'three',
 'minutes',
 'fully_dressed',
 'do',
 'some',
 'schtick',
 'in',
 'the',
 'background',
 'perhaps',
 'three',
 'dozen',
 'men',
 'pass',
 'by',
 'all',
 'naked',
 'white',
 'and',
 'black'

In [8]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab: 0.44 mins


In [9]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))



Time to train the model: 12.68 mins


In [10]:
w2v_model.save("word2vec.model")

In [11]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

189818

In [12]:
# Exporting preprocessed dataset for further steps with replaced bigrams
file_export = df.copy()
file_export[0] = file_export[0].apply(lambda x: ' '.join(bigram[x]))

In [13]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [14]:
file_export[0].to_csv("cleaned_review.csv", index=False)

In [15]:
word_vectors = w2v_model.wv

In [16]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))


In [17]:
model.cluster_centers_

array([[-3.23821567e-02,  2.87262650e-02,  8.04570083e-02,
        -2.16880511e-01,  3.02361751e-02, -7.31453005e-02,
         8.05857639e-03,  1.97015421e-01, -1.35185409e-01,
        -1.30321115e-01,  3.12745553e-02, -1.34941779e-01,
         5.01728609e-02,  5.71345045e-02, -8.39376482e-02,
        -2.25920397e-01,  1.50311380e-01,  1.05084617e-01,
        -1.94330481e-02,  1.18265210e-01, -7.02730020e-02,
         1.75064757e-01, -6.40603506e-02,  6.04359350e-02,
         3.37815531e-02, -2.45680053e-02, -8.25709331e-02,
        -2.30156535e-02,  4.90064924e-02, -6.92149633e-03,
         1.12402394e-01, -2.77165736e-02, -3.06819771e-02,
         4.63786665e-02, -1.87095047e-02, -5.48662081e-02,
         1.44344680e-01, -7.33715005e-02, -4.25305729e-02,
         8.60211206e-02,  1.46294880e-03, -6.28088384e-02,
         1.39743029e-01,  3.53359788e-02,  2.77570057e-02,
         3.56498224e-02,  4.59476117e-02,  1.66229538e-02,
         5.15331307e-02,  8.94209402e-02, -9.59511837e-0

In [51]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)


[('st_ds', 0.8356879353523254),
 ('sceens', 0.8340104818344116),
 ('worth_skipping', 0.8250850439071655),
 ('obvious_nods', 0.819431483745575),
 ('speilberg', 0.8189619779586792),
 ('yessir', 0.8170914053916931),
 ('previous_commentators', 0.8150827288627625),
 ('wildly_imaginative', 0.814156711101532),
 ('netherbeast', 0.8128885626792908),
 ('inundated', 0.8107444047927856)]

In [52]:
positive_cluster_center = model.cluster_centers_[1]
negative_cluster_center = model.cluster_centers_[0]

In [78]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)


[('delatour', 0.883544385433197),
 ('roberts_blossom', 0.8794320225715637),
 ('lindon', 0.8793892860412598),
 ('carole_mathews', 0.8779892921447754),
 ('kyra_helene', 0.8777850866317749),
 ('peewee', 0.8738564252853394),
 ('hammerson', 0.8733760714530945),
 ('dana_elcar', 0.873170018196106),
 ('jodies', 0.8722919225692749),
 ('horse_races', 0.8705471158027649)]

In [53]:
words = pd.DataFrame(word_vectors.index_to_key)

In [54]:
words.columns = ['words']

In [55]:
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])


In [56]:
start = time()

words["cluster"] = words["vectors"].apply(lambda x: model.predict([x.astype('double')]))
print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))


Time to train the model: 63.65 mins


In [57]:
"""vec=[]

for i in range(len(words['vectors'])):
    vec.append(model.predict(words['vectors'][i].reshape(1, -1).astype('float')))"""

"vec=[]\n\nfor i in range(len(words['vectors'])):\n    vec.append(model.predict(words['vectors'][i].reshape(1, -1).astype('float')))"

In [58]:
"""len(vec)"""

'len(vec)'

In [59]:
words

,words,vectors,cluster
0,the,"[-0.21185513, -0.15422396, 0.19057572, -0.1520...",[0]
1,and,"[-0.09514717, 0.018168183, -0.044136487, -0.39...",[0]
2,of,"[-0.30162582, -0.08878651, -0.124232866, -0.29...",[0]
3,to,"[-0.6773318, -0.26445127, 0.24340089, -0.26405...",[0]
4,is,"[-0.641791, -0.35387745, 0.124121964, -0.05242...",[0]
...,...,...,...
93235,paramount_classic,"[0.16848078, 0.07179757, -0.026129965, -0.1165...",[0]
93236,ebert_review,"[0.0346315, 0.14012042, 0.018588578, -0.038789...",[0]
93237,nurse_packer,"[-0.09018444, -0.29930675, 0.11089235, -0.0565...",[1]
93238,neediness,"[-0.058380682, -0.015228618, 0.041239202, -0.3...",[0]


In [60]:
words.cluster = words.cluster.apply(lambda x: x[0])


In [61]:
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [62]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [64]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [65]:
sentiment_map = pd.read_csv("sentiment_dictionary.csv")
sentiment_dict = dict(zip(sentiment_map["words"].values, sentiment_map["sentiment_coeff"].values))

In [66]:
'''file_weighting = unsup_reviews.copy()
file_weighting = pd.DataFrame(file_weighting)'''

'file_weighting = unsup_reviews.copy()\nfile_weighting = pd.DataFrame(file_weighting)'

In [67]:
review = pd.read_csv("cleaned_review.csv")

In [68]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(review["0"].values.astype('U'))
features = pd.Series(tfidf.get_feature_names())
review_tfidf = tfidf.transform(review["0"].values.astype('U'))

C:\Users\raja4\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\raja4\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [69]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], str(x['0']).split()))

In [70]:
review['0']

0        i admit the great majority of films released b...
1        take low_budget inexperienced_actors doubling_...
2        everybody has seen back to the future right wh...
3        doris_day was an icon of beauty in singing and...
4        after series of silly fun_loving movies was bi...
                               ...                        
49995    delightfully awful made by david_giancola guy ...
49996    watching time_chasers it obvious that it was m...
49997    at the beginning we can see members of troma_t...
49998    the movie was incredible ever since saw it in ...
49999    tcm came through by acquiring this wonderful s...
Name: 0, Length: 50000, dtype: object

In [71]:
%%time
replaced_tfidf_scores = review.apply(lambda x: replace_tfidf_words(x, review_tfidf, features), axis=1)

CPU times: total: 10.8 s
Wall time: 14.6 s


In [72]:
def replace_sentiment_words(word, sentiment_dict):
    """
    replacing each word with its associated sentiment score from sentiment dict
    """
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [73]:
replaced_closeness_scores = review["0"].apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), str(x).split())))


In [74]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, review["0"]]).T
replacement_df.columns = ["sentiment_coeff", "tfidf_scores", "review"]
replacement_df["sentiment_rate"] = replacement_df.apply(lambda x: np.array(x.loc["sentiment_coeff"]) @ np.array(x.loc["tfidf_scores"]), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')


In [75]:
replacement_df

,sentiment_coeff,tfidf_scores,review,sentiment_rate,prediction
0,"[0.0969717556831851, 0.0992437521512091, 0.263...","[2.799529422488119, 5.257354190516566, 8.06054...",i admit the great majority of films released b...,58.068170,1
1,"[0.1158965177773626, 0.08701826726418, 0.54519...","[3.2111204026718703, 4.457153013792461, 10.028...",take low_budget inexperienced_actors doubling_...,82.032456,1
2,"[0.0990853241104944, 0.1586218447658028, 0.112...","[5.330593352884969, 3.9433297836124703, 3.0233...",everybody has seen back to the future right wh...,145.001353,1
3,"[0.0764330961170044, 0.1491461881533719, 0.119...","[7.615105664819319, 2.8756240757651703, 1.9254...",doris_day was an icon of beauty in singing and...,61.576902,1
4,"[-0.1340982472710142, 0.1091749086227569, 0.22...","[5.166800755829627, 3.647351465867507, 4.19818...",after series of silly fun_loving movies was bi...,-8.479494,0
...,...,...,...,...,...
49995,"[0.098878489634913, 0.1098077394959636, 0.1428...","[7.204677767369026, 4.026211480838702, 7.38906...",delightfully awful made by david_giancola guy ...,58.590931,1
49996,"[0.1124688754032146, 0.2298015385189905, 0.179...","[3.113804569902385, 9.334891634422286, 2.23652...",watching time_chasers it obvious that it was m...,39.725089,1
49997,"[0.1351095157576171, 0.2637897870587913, 0.120...","[1.8396738171354543, 9.06811575801599, 3.98066...",at the beginning we can see members of troma_t...,150.601327,1
49998,"[0.2637897870587913, 0.1630778173203641, 0.149...","[2.0151368351146646, 1.4918603236549126, 2.875...",the movie was incredible ever since saw it in ...,23.817659,1
